## Applied Data Science Capstone Project

#### Importing the libraries

In [2]:
import requests 
import pandas as pd 
import numpy as np 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

#### Setting up Foursquare Data

In [3]:
ClIENT_ID = 'HWHQ5TRUCJGXTAYGMYLGNCTZTWCBNPIS51DBQ3UOLQRZEDRA' # your Foursquare ID
ClIENT_SECRET = 'ML4ERCN5FJUHA3NGQHFQX402HCSJJWXJ32PLLTGE2JS3IJIF' # your Foursquare Secret
VERSION = '20180505'
LIMIT =30
print('Your credentails:')
print('Foursquare_ID: ' + ClIENT_ID)
print('Foursquare_Secret:' + ClIENT_SECRET)

Your credentails:
Foursquare_ID: HWHQ5TRUCJGXTAYGMYLGNCTZTWCBNPIS51DBQ3UOLQRZEDRA
Foursquare_Secret:ML4ERCN5FJUHA3NGQHFQX402HCSJJWXJ32PLLTGE2JS3IJIF


#### City and Coordinates

In [4]:
city = 'New Delhi'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

28.6141793 77.2022662


#### Searching for Bookstores

In [5]:
search_query = 'Bookstore'
radius = 10000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=HWHQ5TRUCJGXTAYGMYLGNCTZTWCBNPIS51DBQ3UOLQRZEDRA&client_secret=ML4ERCN5FJUHA3NGQHFQX402HCSJJWXJ32PLLTGE2JS3IJIF&ll=28.6141793,77.2022662&v=20180505&query=Bookstore&radius=10000&limit=30'

In [6]:
results = requests.get(url).json()

In [7]:
venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",False,4bffbe2258af952141aaf1b6,"81 Block N, Outer Circle, Connaught Place",IN,New Delhi,India,Barakhamba Rd,2730,"[81 Block N, Outer Circle, Connaught Place (Ba...","[{'label': 'display', 'lat': 28.63134256855394...",28.631343,77.222229,110001,Delhi,Oxford Bookstore,v-1577376070
1,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",False,4ce788eb678aa093731ee3ea,"1st & 2nd Flr., 23, Middle Lane, Khan Market",IN,New Delhi,India,NaN,2891,"[1st & 2nd Flr., 23, Middle Lane, Khan Market,...","[{'label': 'display', 'lat': 28.60019967343879...",28.600200,77.227198,110003,Delhi,Full Circle Bookstore,v-1577376070
2,"[{'id': '4bf58dd8d48988d1dc931735', 'name': 'T...",False,51690773e4b09433491d990e,"81, N-Block, Outer Circle, Connaught Place",IN,New Delhi,India,NaN,2695,"[81, N-Block, Outer Circle, Connaught Place, N...","[{'label': 'display', 'lat': 28.63092047361482...",28.630920,77.222194,110001,Delhi,Cha Bar | चा बार,v-1577376070
3,[],False,4e3d1b327d8b0e961064c873,NaN,IN,NaN,India,NaN,6689,[India],"[{'label': 'display', 'lat': 28.55439, 'lng': ...",28.554390,77.195370,NaN,NaN,Midland bookstore,v-1577376070
4,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",False,4f59e523e4b0a5eaef6df08a,25,IN,New Delhi,India,Janpath Market,2270,"[25 (Janpath Market), New Delhi 110001, Delhi,...","[{'label': 'display', 'lat': 28.62761469932208...",28.627615,77.219743,110001,Delhi,Famous book store,v-1577376070


#### Cleaning the Bookstore Dataframe

In [8]:
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Oxford Bookstore,Bookstore,"81 Block N, Outer Circle, Connaught Place",IN,New Delhi,India,Barakhamba Rd,2730,"[81 Block N, Outer Circle, Connaught Place (Ba...","[{'label': 'display', 'lat': 28.63134256855394...",28.631343,77.222229,110001,Delhi,4bffbe2258af952141aaf1b6
1,Full Circle Bookstore,Bookstore,"1st & 2nd Flr., 23, Middle Lane, Khan Market",IN,New Delhi,India,NaN,2891,"[1st & 2nd Flr., 23, Middle Lane, Khan Market,...","[{'label': 'display', 'lat': 28.60019967343879...",28.600200,77.227198,110003,Delhi,4ce788eb678aa093731ee3ea
2,Cha Bar | चा बार,Tea Room,"81, N-Block, Outer Circle, Connaught Place",IN,New Delhi,India,NaN,2695,"[81, N-Block, Outer Circle, Connaught Place, N...","[{'label': 'display', 'lat': 28.63092047361482...",28.630920,77.222194,110001,Delhi,51690773e4b09433491d990e
3,Midland bookstore,None,NaN,IN,NaN,India,NaN,6689,[India],"[{'label': 'display', 'lat': 28.55439, 'lng': ...",28.554390,77.195370,NaN,NaN,4e3d1b327d8b0e961064c873
4,Famous book store,Bookstore,25,IN,New Delhi,India,Janpath Market,2270,"[25 (Janpath Market), New Delhi 110001, Delhi,...","[{'label': 'display', 'lat': 28.62761469932208...",28.627615,77.219743,110001,Delhi,4f59e523e4b0a5eaef6df08a


In [9]:
#removing unnecessary data
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress', 'labeledLatLngs', 'state', 'id'], axis=1)
clean_dataframe2

,name,categories,address,lat,lng,postalCode
0,Oxford Bookstore,Bookstore,"81 Block N, Outer Circle, Connaught Place",28.631343,77.222229,110001
1,Full Circle Bookstore,Bookstore,"1st & 2nd Flr., 23, Middle Lane, Khan Market",28.600200,77.227198,110003
2,Cha Bar | चा बार,Tea Room,"81, N-Block, Outer Circle, Connaught Place",28.630920,77.222194,110001
3,Midland bookstore,None,NaN,28.554390,77.195370,NaN
4,Famous book store,Bookstore,25,28.627615,77.219743,110001
5,Lakshmi Book Store,Bookstore,18 Janpath Bhawan,28.629590,77.218506,110001
6,Bahrisons Bookstore,Bookstore,Dlf Saket,28.527848,77.206437,NaN
7,Half Price Bookstore,Bookstore,Select Citywalk,28.529231,77.219490,NaN
8,Wordsworth bookstore,Bookstore,NaN,28.630448,77.081716,NaN
9,Delhi Book Store,Bookstore,Darya Ganj,28.639880,77.250610,NaN


In [10]:
#removing blank values
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_dataframe3

,name,categories,address,lat,lng,postalCode
0,Oxford Bookstore,Bookstore,"81 Block N, Outer Circle, Connaught Place",28.631343,77.222229,110001
1,Full Circle Bookstore,Bookstore,"1st & 2nd Flr., 23, Middle Lane, Khan Market",28.600200,77.227198,110003
2,Cha Bar | चा बार,Tea Room,"81, N-Block, Outer Circle, Connaught Place",28.630920,77.222194,110001
4,Famous book store,Bookstore,25,28.627615,77.219743,110001
5,Lakshmi Book Store,Bookstore,18 Janpath Bhawan,28.629590,77.218506,110001


In [11]:
#removing all values whose category is not bookstore
array= ['Bookstore']
book_dataframe= clean_dataframe3.loc[clean_dataframe3['categories'].isin(array)]
book_dataframe

,name,categories,address,lat,lng,postalCode
0,Oxford Bookstore,Bookstore,"81 Block N, Outer Circle, Connaught Place",28.631343,77.222229,110001
1,Full Circle Bookstore,Bookstore,"1st & 2nd Flr., 23, Middle Lane, Khan Market",28.600200,77.227198,110003
4,Famous book store,Bookstore,25,28.627615,77.219743,110001
5,Lakshmi Book Store,Bookstore,18 Janpath Bhawan,28.629590,77.218506,110001


In [12]:
#removing duplicate values
df_book = book_dataframe.drop_duplicates(subset='name', keep="first")
df_book

,name,categories,address,lat,lng,postalCode
0,Oxford Bookstore,Bookstore,"81 Block N, Outer Circle, Connaught Place",28.631343,77.222229,110001
1,Full Circle Bookstore,Bookstore,"1st & 2nd Flr., 23, Middle Lane, Khan Market",28.600200,77.227198,110003
4,Famous book store,Bookstore,25,28.627615,77.219743,110001
5,Lakshmi Book Store,Bookstore,18 Janpath Bhawan,28.629590,77.218506,110001


#### Searching for Cafes

In [13]:
search_query = 'Cafe'
radius = 10000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=HWHQ5TRUCJGXTAYGMYLGNCTZTWCBNPIS51DBQ3UOLQRZEDRA&client_secret=ML4ERCN5FJUHA3NGQHFQX402HCSJJWXJ32PLLTGE2JS3IJIF&ll=28.6141793,77.2022662&v=20180505&query=Cafe&radius=10000&limit=30'

In [14]:
cresults = requests.get(url).json()

In [15]:
venues = cresults['response']['venues']

caf_dataframe = json_normalize(venues)
caf_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,4b701cbaf964a5209b072de3,68A,IN,New Delhi,India,Khan Market,2898,"[68A (Khan Market), New Delhi 110048, Delhi, I...","[{'label': 'display', 'lat': 28.60068629868818...",28.600686,77.227636,NaN,110048,Delhi,The Big Chill Cafe,v-1577376084
1,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,50bcb63ee4b0bedc1d3c28f7,C Block,IN,Delhi,India,CP,2852,"[C Block (CP), Delhi, Delhi, India]","[{'label': 'display', 'lat': 28.6342949764577,...",28.634295,77.220345,NaN,NaN,Delhi,Cafe Coffee Day Square,v-1577376084
2,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,5117d099e4b0bae1afcea5c6,Inside Regal Building,IN,New Delhi,India,"Connaught Place, Near Regal Cinema",2365,"[Inside Regal Building (Connaught Place, Near ...","[{'label': 'display', 'lat': 28.631054, 'lng':...",28.631054,77.216978,NaN,110001,Delhi,Café Coffee Day,v-1577376084
3,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,5279c495498e3592ada19181,NaN,IN,NaN,India,NaN,2414,[India],"[{'label': 'display', 'lat': 28.63095845041277...",28.630958,77.217929,NaN,NaN,NaN,Cafe Coffee Day,v-1577376084
4,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,4d6b798b2ea9b1f78cf9d028,Outer Circle Connaught Place,IN,New Delhi,India,N Blk,2695,"[Outer Circle Connaught Place (N Blk), New Del...","[{'label': 'display', 'lat': 28.63133963244659...",28.631340,77.221726,NaN,NaN,Delhi,Cafe Coffee Day,v-1577376084


#### Cleaning the Cafes Dataframe

In [16]:
caf_clean_columns = ['name', 'categories'] + [col for col in caf_dataframe.columns if col.startswith('location.')]+ ['id']
clean_caf_dataframe = caf_dataframe.loc[:,caf_clean_columns]

def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

clean_caf_dataframe['categories'] = clean_caf_dataframe.apply(get_category_type, axis=1)

clean_caf_dataframe.columns = [column.split('.')[-1] for column in clean_caf_dataframe.columns]

clean_caf_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,The Big Chill Cafe,Italian Restaurant,68A,IN,New Delhi,India,Khan Market,2898,"[68A (Khan Market), New Delhi 110048, Delhi, I...","[{'label': 'display', 'lat': 28.60068629868818...",28.600686,77.227636,NaN,110048,Delhi,4b701cbaf964a5209b072de3
1,Cafe Coffee Day Square,Coffee Shop,C Block,IN,Delhi,India,CP,2852,"[C Block (CP), Delhi, Delhi, India]","[{'label': 'display', 'lat': 28.6342949764577,...",28.634295,77.220345,NaN,NaN,Delhi,50bcb63ee4b0bedc1d3c28f7
2,Café Coffee Day,Coffee Shop,Inside Regal Building,IN,New Delhi,India,"Connaught Place, Near Regal Cinema",2365,"[Inside Regal Building (Connaught Place, Near ...","[{'label': 'display', 'lat': 28.631054, 'lng':...",28.631054,77.216978,NaN,110001,Delhi,5117d099e4b0bae1afcea5c6
3,Cafe Coffee Day,Coffee Shop,NaN,IN,NaN,India,NaN,2414,[India],"[{'label': 'display', 'lat': 28.63095845041277...",28.630958,77.217929,NaN,NaN,NaN,5279c495498e3592ada19181
4,Cafe Coffee Day,Café,Outer Circle Connaught Place,IN,New Delhi,India,N Blk,2695,"[Outer Circle Connaught Place (N Blk), New Del...","[{'label': 'display', 'lat': 28.63133963244659...",28.631340,77.221726,NaN,NaN,Delhi,4d6b798b2ea9b1f78cf9d028


In [17]:
#dropping unnecessary values
clean_caf_dataframe2= clean_caf_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'state', 'neighborhood', 'postalCode', 'id'], axis=1)
clean_caf_dataframe2

,name,categories,address,lat,lng
0,The Big Chill Cafe,Italian Restaurant,68A,28.600686,77.227636
1,Cafe Coffee Day Square,Coffee Shop,C Block,28.634295,77.220345
2,Café Coffee Day,Coffee Shop,Inside Regal Building,28.631054,77.216978
3,Cafe Coffee Day,Coffee Shop,NaN,28.630958,77.217929
4,Cafe Coffee Day,Café,Outer Circle Connaught Place,28.631340,77.221726
5,Cafe,Café,CMYK,28.586478,77.226498
6,Cafe Coffee Day,Café,"Pallavi AWWA Complex, Chanakyapuri",28.595247,77.171954
7,Mysore Cafe,Tea Room,NaN,28.608261,77.201475
8,Cafe Coffee Day,Café,Connaught Place,28.633841,77.217909
9,The Beer Cafe,Beer Garden,Scindia House,28.629139,77.219603


In [18]:
#getting rid of blank values
clean_caf_dataframe3 = clean_caf_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_caf_dataframe3

,name,categories,address,lat,lng
0,The Big Chill Cafe,Italian Restaurant,68A,28.600686,77.227636
1,Cafe Coffee Day Square,Coffee Shop,C Block,28.634295,77.220345
2,Café Coffee Day,Coffee Shop,Inside Regal Building,28.631054,77.216978
4,Cafe Coffee Day,Café,Outer Circle Connaught Place,28.631340,77.221726
5,Cafe,Café,CMYK,28.586478,77.226498
6,Cafe Coffee Day,Café,"Pallavi AWWA Complex, Chanakyapuri",28.595247,77.171954
8,Cafe Coffee Day,Café,Connaught Place,28.633841,77.217909
9,The Beer Cafe,Beer Garden,Scindia House,28.629139,77.219603
10,Boombox Cafe,Lounge,"Rajiv Chowk, F Block",28.631275,77.220041
11,Café Nirvana,Café,Main Bazar Rd.,28.641334,77.214683


In [19]:
#keeping only those values whose category is cafe
df_caf = clean_caf_dataframe3[clean_caf_dataframe3.categories == 'Café']
df_caf

,name,categories,address,lat,lng
4,Cafe Coffee Day,Café,Outer Circle Connaught Place,28.631340,77.221726
5,Cafe,Café,CMYK,28.586478,77.226498
6,Cafe Coffee Day,Café,"Pallavi AWWA Complex, Chanakyapuri",28.595247,77.171954
8,Cafe Coffee Day,Café,Connaught Place,28.633841,77.217909
11,Café Nirvana,Café,Main Bazar Rd.,28.641334,77.214683
12,1 Cafe Lounge,Café,"Outer Circle, Connaught Place",28.633142,77.222518
21,Triveni Terrace Cafe,Café,Triveni Kala Sangam,28.627295,77.232677
22,Cafe Coffee Day,Café,Delhi Airport Metro Express,28.617903,77.188474
23,The Pool Cafe @ The Lodhi Hotel,Café,The Lodhi Hotel,28.591559,77.238278
24,Cafe Delhi Heights,Café,"68/1, Janpath, Atul Grove Road, Janpath, Conna...",28.627842,77.219860


#### Searching for Theatres

In [20]:
search_query = 'Theatre'
radius = 10000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=HWHQ5TRUCJGXTAYGMYLGNCTZTWCBNPIS51DBQ3UOLQRZEDRA&client_secret=ML4ERCN5FJUHA3NGQHFQX402HCSJJWXJ32PLLTGE2JS3IJIF&ll=28.6141793,77.2022662&v=20180505&query=Theatre&radius=10000&limit=30'

In [21]:
tresults = requests.get(url).json()

In [22]:
venues = tresults['response']['venues']

th_dataframe = json_normalize(venues)
th_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d137941735', 'name': 'T...",False,4d0a11f8e6f83704ef085e87,"11-B, Baba Kharak Singh Marg",IN,New Delhi,India,Next to RML Hospital exit no. 5,1283,"[11-B, Baba Kharak Singh Marg (Next to RML Hos...","[{'label': 'display', 'lat': 28.62566512551797...",28.625665,77.203379,110001,Delhi,Akshara Theatre | अक्षरा थिएटर,v-1577376088,NaN
1,"[{'id': '5032792091d4c4b30a586d5c', 'name': 'C...",False,516174b1e4b0878cfbc6f827,NaN,IN,NaN,India,NaN,1529,[India],"[{'label': 'display', 'lat': 28.625151, 'lng':...",28.625151,77.192840,NaN,NaN,Surtal(Open Air Theatre) Talkatora,v-1577376088,NaN
2,"[{'id': '4bf58dd8d48988d137941735', 'name': 'T...",False,507ab579e4b0753d5a388f68,NaN,IN,NaN,India,NaN,2929,[India],"[{'label': 'display', 'lat': 28.61403127161683...",28.614031,77.232244,NaN,NaN,Meghdoot Theatre,v-1577376088,NaN
3,"[{'id': '4bf58dd8d48988d137941735', 'name': 'T...",False,5098fb675262fd71fdd1564c,COPERNICUS MARG,IN,Delhi,India,DOORDARSHAN BHAWAN,2990,"[COPERNICUS MARG (DOORDARSHAN BHAWAN), Delhi 1...","[{'label': 'display', 'lat': 28.6218039088934,...",28.621804,77.231612,110001,Delhi,little theatre group,v-1577376088,NaN
4,"[{'id': '4bf58dd8d48988d137941735', 'name': 'T...",False,4f4dfec0e4b088c4aed32436,Nav shakti school,IN,NaN,India,ITO,4320,"[Nav shakti school (ITO), India]","[{'label': 'display', 'lat': 28.63281033753914...",28.632810,77.241051,NaN,NaN,asmita theatre,v-1577376088,NaN


#### Cleaning the Theatres Dataframe

In [23]:
th_clean_columns = ['name', 'categories'] + [col for col in th_dataframe.columns if col.startswith('location.')]+ ['id']
clean_th_dataframe = th_dataframe.loc[:,th_clean_columns]

def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

clean_th_dataframe['categories'] = clean_th_dataframe.apply(get_category_type, axis=1)

clean_th_dataframe.columns = [column.split('.')[-1] for column in clean_th_dataframe.columns]

clean_th_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Akshara Theatre | अक्षरा थिएटर,Theater,"11-B, Baba Kharak Singh Marg",IN,New Delhi,India,Next to RML Hospital exit no. 5,1283,"[11-B, Baba Kharak Singh Marg (Next to RML Hos...","[{'label': 'display', 'lat': 28.62566512551797...",28.625665,77.203379,110001,Delhi,4d0a11f8e6f83704ef085e87
1,Surtal(Open Air Theatre) Talkatora,Concert Hall,NaN,IN,NaN,India,NaN,1529,[India],"[{'label': 'display', 'lat': 28.625151, 'lng':...",28.625151,77.192840,NaN,NaN,516174b1e4b0878cfbc6f827
2,Meghdoot Theatre,Theater,NaN,IN,NaN,India,NaN,2929,[India],"[{'label': 'display', 'lat': 28.61403127161683...",28.614031,77.232244,NaN,NaN,507ab579e4b0753d5a388f68
3,little theatre group,Theater,COPERNICUS MARG,IN,Delhi,India,DOORDARSHAN BHAWAN,2990,"[COPERNICUS MARG (DOORDARSHAN BHAWAN), Delhi 1...","[{'label': 'display', 'lat': 28.6218039088934,...",28.621804,77.231612,110001,Delhi,5098fb675262fd71fdd1564c
4,asmita theatre,Theater,Nav shakti school,IN,NaN,India,ITO,4320,"[Nav shakti school (ITO), India]","[{'label': 'display', 'lat': 28.63281033753914...",28.632810,77.241051,NaN,NaN,4f4dfec0e4b088c4aed32436


In [24]:
#dropping unnecessary values
clean_th_dataframe2= clean_th_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'postalCode', 'state', 'id'], axis=1)
clean_th_dataframe2

,name,categories,address,lat,lng
0,Akshara Theatre | अक्षरा थिएटर,Theater,"11-B, Baba Kharak Singh Marg",28.625665,77.203379
1,Surtal(Open Air Theatre) Talkatora,Concert Hall,NaN,28.625151,77.192840
2,Meghdoot Theatre,Theater,NaN,28.614031,77.232244
3,little theatre group,Theater,COPERNICUS MARG,28.621804,77.231612
4,asmita theatre,Theater,Nav shakti school,28.632810,77.241051
5,"Surgery Operation Theatre, LNJP Hospital",College Academic Building,NaN,28.638393,77.236830
6,Open Air Theatre,Indie Movie Theater,Taurus,28.598426,77.141919
7,Actor Factor Theatre Company,Theater,NaN,28.548530,77.211810
8,The hoshruba Theatre Company,Theater,209 A Peacock Lane,28.547016,77.213792
9,ARIA Music & Theatre Conservatory,Music Venue,C-52 A,28.646652,77.122512


In [25]:
#removing blank values
df_th = clean_th_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_th

,name,categories,address,lat,lng
0,Akshara Theatre | अक्षरा थिएटर,Theater,"11-B, Baba Kharak Singh Marg",28.625665,77.203379
3,little theatre group,Theater,COPERNICUS MARG,28.621804,77.231612
4,asmita theatre,Theater,Nav shakti school,28.632810,77.241051
6,Open Air Theatre,Indie Movie Theater,Taurus,28.598426,77.141919
8,The hoshruba Theatre Company,Theater,209 A Peacock Lane,28.547016,77.213792
9,ARIA Music & Theatre Conservatory,Music Venue,C-52 A,28.646652,77.122512
10,Open Air Theatre,College Theater,JNU UNIVERSITY,28.548194,77.168010
12,Oddbird Theatre,Indie Theater,"Dhan Mill Compound, 100 Foot Road, SSN Marg, P...",28.506228,77.183645
13,The Atre Yoga Studio,Yoga Studio,"252-A, Nanak Bhavan",28.548267,77.211383


#### Searching for Parks

In [26]:
search_query = 'Park'
radius = 10000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=HWHQ5TRUCJGXTAYGMYLGNCTZTWCBNPIS51DBQ3UOLQRZEDRA&client_secret=ML4ERCN5FJUHA3NGQHFQX402HCSJJWXJ32PLLTGE2JS3IJIF&ll=28.6141793,77.2022662&v=20180505&query=Park&radius=10000&limit=30'

In [27]:
presults = requests.get(url).json()

In [28]:
venues = presults['response']['venues']

park_dataframe = json_normalize(venues)
park_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4b80e458f964a520bf9030e3,15 Parliament St,IN,New Delhi,India,NaN,2135,"[15 Parliament St, New Delhi 110 001, Delhi, I...","[{'label': 'display', 'lat': 28.62873117053961...",28.628731,77.216507,NaN,110 001,Delhi,The Park Hotel,v-1577376094
1,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,5bd99a5d8ad62e002cc35a90,NaN,IN,New Delhi,India,NaN,2682,"[New Delhi 110001, Delhi, India]","[{'label': 'display', 'lat': 28.632954, 'lng':...",28.632954,77.219470,NaN,110001,Delhi,Central Park - New Delhi,v-1577376094
2,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4e18a5cd81309771577a1f9a,Pandara Park,IN,New Delhi,India,Lodi Estate,3100,"[Pandara Park (Lodi Estate), New Delhi 110003,...","[{'label': 'display', 'lat': 28.60352005138293...",28.603520,77.231580,NaN,110003,Delhi,Pandara Rd Local Park,v-1577376094
3,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4ce89bd9e888f04dbda7436b,Chanakyapuri,IN,New Delhi,India,NaN,2655,"[Chanakyapuri, New Delhi, Delhi, India]","[{'label': 'display', 'lat': 28.59179820602936...",28.591798,77.192860,NaN,NaN,Delhi,Nehru Park | नेहरू पार्क (Nehru Park),v-1577376094
4,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,5dac8f1928f17a000861f133,NaN,IN,ニューデリー,India,NaN,2122,"[ニューデリー 110001, デリー, India]","[{'label': 'display', 'lat': 28.629497, 'lng':...",28.629497,77.215203,NaN,110001,デリー,Pracheen Hanuman Mandir Park,v-1577376094


#### Cleaning the Parks Dataframe

In [29]:
park_clean_columns = ['name', 'categories'] + [col for col in park_dataframe.columns if col.startswith('location.')]+ ['id']
clean_park_dataframe = park_dataframe.loc[:,park_clean_columns]

def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

clean_park_dataframe['categories'] = clean_park_dataframe.apply(get_category_type, axis=1)

clean_park_dataframe.columns = [column.split('.')[-1] for column in clean_park_dataframe.columns]

clean_park_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,The Park Hotel,Hotel,15 Parliament St,IN,New Delhi,India,NaN,2135,"[15 Parliament St, New Delhi 110 001, Delhi, I...","[{'label': 'display', 'lat': 28.62873117053961...",28.628731,77.216507,NaN,110 001,Delhi,4b80e458f964a520bf9030e3
1,Central Park - New Delhi,Park,NaN,IN,New Delhi,India,NaN,2682,"[New Delhi 110001, Delhi, India]","[{'label': 'display', 'lat': 28.632954, 'lng':...",28.632954,77.219470,NaN,110001,Delhi,5bd99a5d8ad62e002cc35a90
2,Pandara Rd Local Park,Park,Pandara Park,IN,New Delhi,India,Lodi Estate,3100,"[Pandara Park (Lodi Estate), New Delhi 110003,...","[{'label': 'display', 'lat': 28.60352005138293...",28.603520,77.231580,NaN,110003,Delhi,4e18a5cd81309771577a1f9a
3,Nehru Park | नेहरू पार्क (Nehru Park),Park,Chanakyapuri,IN,New Delhi,India,NaN,2655,"[Chanakyapuri, New Delhi, Delhi, India]","[{'label': 'display', 'lat': 28.59179820602936...",28.591798,77.192860,NaN,NaN,Delhi,4ce89bd9e888f04dbda7436b
4,Pracheen Hanuman Mandir Park,Park,NaN,IN,ニューデリー,India,NaN,2122,"[ニューデリー 110001, デリー, India]","[{'label': 'display', 'lat': 28.629497, 'lng':...",28.629497,77.215203,NaN,110001,デリー,5dac8f1928f17a000861f133


In [30]:
#dropping unnecessary values
clean_park_dataframe2= clean_park_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'postalCode', 'state', 'id'], axis=1)
clean_park_dataframe2

,name,categories,address,lat,lng
0,The Park Hotel,Hotel,15 Parliament St,28.628731,77.216507
1,Central Park - New Delhi,Park,NaN,28.632954,77.219470
2,Pandara Rd Local Park,Park,Pandara Park,28.603520,77.231580
3,Nehru Park | नेहरू पार्क (Nehru Park),Park,Chanakyapuri,28.591798,77.192860
4,Pracheen Hanuman Mandir Park,Park,NaN,28.629497,77.215203
5,National Zoological Park | राष्ट्रीय प्राणी उद...,Zoo,Off Sunder Nagar,28.607453,77.241686
6,Triangle Park,Park,Pandara Road,28.604606,77.228823
7,Gulmohar Park Road,Park,177 Adhar Shila,28.559251,77.213404
8,Park Inn by Radisson New Delhi Lajpat Nagar,Hotel,"1&2, Ring Road, Lajpat Nagar 4,",28.563750,77.234810
9,Hotel Kingston Park New Delhi,Hotel,8/5 WEA Karol Bagh,28.645230,77.192467


In [31]:
#removing blank values
clean_park_dataframe3 = clean_park_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_park_dataframe3

,name,categories,address,lat,lng
0,The Park Hotel,Hotel,15 Parliament St,28.628731,77.216507
2,Pandara Rd Local Park,Park,Pandara Park,28.603520,77.231580
3,Nehru Park | नेहरू पार्क (Nehru Park),Park,Chanakyapuri,28.591798,77.192860
5,National Zoological Park | राष्ट्रीय प्राणी उद...,Zoo,Off Sunder Nagar,28.607453,77.241686
6,Triangle Park,Park,Pandara Road,28.604606,77.228823
7,Gulmohar Park Road,Park,177 Adhar Shila,28.559251,77.213404
8,Park Inn by Radisson New Delhi Lajpat Nagar,Hotel,"1&2, Ring Road, Lajpat Nagar 4,",28.563750,77.234810
9,Hotel Kingston Park New Delhi,Hotel,8/5 WEA Karol Bagh,28.645230,77.192467
10,Deer Park,Park,Safdarjung Enclave,28.558191,77.193814
11,Panchsheel Park | पंचशील पार्क,Park,Josip Broz Tito Marg,28.553746,77.228652


In [32]:
#keeping only those values whose category is park
df_park = clean_park_dataframe3[clean_park_dataframe3.categories == 'Park']
df_park

,name,categories,address,lat,lng
2,Pandara Rd Local Park,Park,Pandara Park,28.603520,77.231580
3,Nehru Park | नेहरू पार्क (Nehru Park),Park,Chanakyapuri,28.591798,77.192860
6,Triangle Park,Park,Pandara Road,28.604606,77.228823
7,Gulmohar Park Road,Park,177 Adhar Shila,28.559251,77.213404
10,Deer Park,Park,Safdarjung Enclave,28.558191,77.193814
11,Panchsheel Park | पंचशील पार्क,Park,Josip Broz Tito Marg,28.553746,77.228652
16,Thyagraj Park,Park,Thyagraj Park,28.578247,77.214983
18,Central Park,Park,Connaught Place,28.632614,77.219299
28,Pandara Park,Park,Pandara Park,28.606781,77.233019


## Generate map to visualize Bookstores, Cafes, Theatres and Parks to see how they Cluster

In [33]:
#concatenating all dataframes into one
proj_neighbourhood_df = pd.concat([df_park, df_book, df_caf, df_th], sort=True)
proj_neighbourhood_df

,address,categories,lat,lng,name,postalCode
2,Pandara Park,Park,28.603520,77.231580,Pandara Rd Local Park,NaN
3,Chanakyapuri,Park,28.591798,77.192860,Nehru Park | नेहरू पार्क (Nehru Park),NaN
6,Pandara Road,Park,28.604606,77.228823,Triangle Park,NaN
7,177 Adhar Shila,Park,28.559251,77.213404,Gulmohar Park Road,NaN
10,Safdarjung Enclave,Park,28.558191,77.193814,Deer Park,NaN
11,Josip Broz Tito Marg,Park,28.553746,77.228652,Panchsheel Park | पंचशील पार्क,NaN
16,Thyagraj Park,Park,28.578247,77.214983,Thyagraj Park,NaN
18,Connaught Place,Park,28.632614,77.219299,Central Park,NaN
28,Pandara Park,Park,28.606781,77.233019,Pandara Park,NaN
0,"81 Block N, Outer Circle, Connaught Place",Bookstore,28.631343,77.222229,Oxford Bookstore,110001


In [34]:
proj_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(proj_neighbourhood_df['lat'], proj_neighbourhood_df['lng'], 
                                           proj_neighbourhood_df['name'], proj_neighbourhood_df['categories'],\
                                               proj_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(proj_map)  
    
proj_map